In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load historical sales data
sales_data = pd.read_csv('drug_sales.csv')

# Melt the data to long format for easier processing
sales_data_long = pd.melt(sales_data, id_vars=['Drug'], var_name='Season', value_name='Sales')

# Fit LabelEncoder on all possible seasons and drugs
label_encoder_season = LabelEncoder()
label_encoder_drug = LabelEncoder()

all_seasons = sales_data_long['Season'].unique()
label_encoder_season.fit(all_seasons)

all_drugs = sales_data_long['Drug'].unique()
label_encoder_drug.fit(all_drugs)

sales_data_long['Season_Encoded'] = label_encoder_season.transform(sales_data_long['Season'])
sales_data_long['Drug_Encoded'] = label_encoder_drug.transform(sales_data_long['Drug'])

# Define features and target
features = ['Season_Encoded']
target = 'Sales'

# Feature and target variables
X = sales_data_long[features]
y = sales_data_long[target]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics (only MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate percentage metrics
mean_sales = y_test.mean()
mae_percentage = (mae / mean_sales) * 100

print(f"Mean Absolute Error (MAE): {mae_percentage:.2f}% of mean sales")

# Function to evaluate prediction quality
def evaluate_prediction_quality(mae_percentage, threshold=10):
    if mae_percentage <= threshold:
        return "Good"
    else:
        return "Needs Improvement"

# Predict sales for each drug based on the given season
def predict_sales_for_drugs(season):
    # Ensure the season is valid
    if season not in label_encoder_season.classes_:
        raise ValueError(f"Invalid season name: {season}. Choose from {', '.join(label_encoder_season.classes_)}")
    
    season_encoded = label_encoder_season.transform([season])[0]
    
    # Create a DataFrame for all drugs with the input season
    input_df = pd.DataFrame({
        'Season_Encoded': [season_encoded] * len(all_drugs),
        'Drug_Encoded': label_encoder_drug.transform(all_drugs)
    })
    
    # Scale the input features
    input_scaled = scaler.transform(input_df[['Season_Encoded']])
    predicted_sales = model.predict(input_scaled)
    
    # Return predicted sales and drug names
    return pd.DataFrame({
        'Drug': all_drugs,
        'Predicted_Sales': predicted_sales
    })

# Calculate safety stock levels
def calculate_safety_stock(drug):
    historical_data = sales_data_long[sales_data_long['Drug'] == drug]
    
    if historical_data.empty:
        return None
    
    avg_sales = historical_data['Sales'].mean()
    std_sales = historical_data['Sales'].std()
    
    # Example safety factor
    safety_stock = std_sales * 1.5
    return avg_sales + safety_stock

# Main execution block
season = input("Enter the season (Winter, Spring, Summer, Autumn, Monsoon): ")

# Predict sales for each drug
predicted_sales_df = predict_sales_for_drugs(season)

# Calculate safety stock levels based on the predicted sales
safety_stock_levels = {}
for drug in predicted_sales_df['Drug']:
    safety_stock_level = calculate_safety_stock(drug)
    
    if safety_stock_level is not None:
        safety_stock_levels[drug] = safety_stock_level

# Evaluate prediction quality
prediction_quality = evaluate_prediction_quality(mae_percentage)

# Print stock levels for each drug and prediction quality
print("\nRecommended Safety Stock Levels:")
for drug, level in safety_stock_levels.items():
    if not pd.isna(level):  # Check if level is not NaN
        print(f'Drug: {drug}, Recommended Safety Stock Level: {level:.2f}')

print(f"\nPrediction Quality: {prediction_quality} (based on MAE percentage)")






Mean Absolute Error (MAE): 30.03% of mean sales

Recommended Safety Stock Levels:
Drug: Paracetamol, Recommended Safety Stock Level: 536.85
Drug: Ibuprofen, Recommended Safety Stock Level: 333.72
Drug: Aspirin, Recommended Safety Stock Level: 283.72
Drug: Cetirizine, Recommended Safety Stock Level: 201.77
Drug: Amoxicillin, Recommended Safety Stock Level: 483.72
Drug: Azithromycin, Recommended Safety Stock Level: 536.85
Drug: Metformin, Recommended Safety Stock Level: 436.85
Drug: Amlodipine, Recommended Safety Stock Level: 383.72
Drug: Atorvastatin, Recommended Safety Stock Level: 636.85
Drug: Omeprazole, Recommended Safety Stock Level: 283.72
Drug: Losartan, Recommended Safety Stock Level: 333.72
Drug: Levothyroxine, Recommended Safety Stock Level: 436.85
Drug: Clopidogrel, Recommended Safety Stock Level: 233.72
Drug: Doxycycline, Recommended Safety Stock Level: 183.72
Drug: Lisinopril, Recommended Safety Stock Level: 283.72
Drug: Metoprolol, Recommended Safety Stock Level: 333.72
Dr